In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [ ]:
height = 64
width = 128
size = height*width
path = "..\..\datasets\\UNSW_NB15"
csv_file = path + "\\unswnb15_img_serialized_5_nonshuffled.csv"

In [ ]:
df = pd.read_csv(path+"\\UNSW_converted_data.csv")

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.protocol.unique()

In [ ]:
len(df.protocol.unique())

In [ ]:
indices = list(df.protocol.unique())

In [ ]:
indices.index("tcp")

In [ ]:
# move tcp to end of list
indices.remove("tcp")
indices.append("tcp")
indices.index("tcp")

In [ ]:
indices.index("udp")

In [ ]:
# move udp to start of list
indices.remove("udp")
indices = ["udp"] + indices
indices.index("udp")

In [ ]:
df.protocol = df.protocol.apply(lambda x: indices.index(x))

In [ ]:
df.protocol.unique()

In [ ]:
df.label.unique()

In [ ]:
len(df.label.unique())

In [ ]:
df = df.to_numpy()
df.shape, df[0, -1]

In [ ]:
# normalize payload, ttl, total_len
for col_index in range(df.shape[1]-1):
    column = df[:, col_index]
    column_normalized = (column - np.min(column)) / (np.max(column) - np.min(column) + 0.1)
    df[:, col_index] = column_normalized

In [ ]:
def convert_dataset_to_grayscale_image(arr):
    for idx in range(len(arr) - 5):
        batch = arr[idx: idx + 5, :-1]
        label = arr[idx + 5, -1]
        print(batch.shape, label)
        data = np.concatenate(batch)
        print(data.shape)
        
        data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        print(data.shape)
        data = data.reshape(height, width).astype('float64')
        print(data.shape)
        
        plt.imshow(data, cmap='gray')
        plt.axis('off')  # Remove axes
        plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_grayscale_image(df)

In [ ]:
def convert_dataset_to_rgb_image(arr):
    for idx in range(len(arr) - 5):
        batch = arr[idx: idx + 5, :-1]
        label = arr[idx + 5, -1]
        print(batch.shape, label)
        data = np.concatenate(batch)
        print(data.shape)
        
        data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
        data = data.reshape(height, width)
        
        channel_1 = data.astype('float64')
        print(channel_1.shape)
        channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
        print(channel_2.shape)
        channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
        print(channel_3.shape)
        img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
        print(img.shape)
        
        plt.imshow(img)
        plt.axis('off')  # Remove axes
        plt.show()
        
        break
    print("DONE")
        
convert_dataset_to_rgb_image(df)

In [ ]:
with open(csv_file, 'w') as f:
    np.savetxt(f, [np.array(["file_name", "label"])], delimiter=',', fmt='%s')

def convert_dataset_to_image(arr):
    with open(csv_file, 'a') as f:
        for idx in range(len(arr) - 5):
            batch = arr[idx: idx + 5, :-1]
            label = arr[idx + 5, -1]
            data = np.concatenate(batch)
            if idx % 1000 == 0:
                print(idx, label)
            
            data = np.pad(data, pad_width=int((size-len(data))/2), constant_values=0)
            data = data.reshape(height, width)
            
            channel_1 = data.astype('float64')
            channel_2 = np.rot90(channel_1, k=2).reshape(height, width)
            channel_3 = np.rot90(channel_2, k=2).reshape(height, width)
            img = np.stack((channel_1, channel_2, channel_3)).transpose((1, 2, 0))
            
            file_name = f"unswnb15_{idx}.png"
            cv2.imwrite(path+"\image_serialized_5_non_shuffled\\"+file_name, img*255)
            
            log = np.array([file_name, label])
            np.savetxt(f, [log], delimiter=',', fmt='%s')
    print("DONE")
        

convert_dataset_to_image(df)